# 5장. 구조적 API 기본 연산
- 구조적 API의 아키텍처 개념에서 벗어나 DataFrame과 DataFrame의 데이터를 다루는 기능을 소개
  - DataFrame의 기본 기능을 중점적으로 다룸
- DataFrame은 Row 타입의 레코드(테이블의 로우 같은)와 각 레코드에 수행할 연산 표현식을 나타내는 여러 컬럼(스프레드시트의 컬럼 같은)으로 구성됨
  - 스키마 : 각 컬럼명과 데이터 타입을 정의
  - DataFrame의 파티셔닝 : DataFrame이나 Dataset이 클러스터에서 물리적으로 배치되는 형태를 정의
    - 파티셔닝 스키마 : 파티션을 배치하는 방법을 정의함
    - 파티셔닝의 분할 기준 : 특정 컬럼이나 비결정론적 값(매번 변하는 값)을 기반으로 설정할 수 있음


In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sample").master("local[*]").getOrCreate()

In [6]:
#DataFrame을 생성함
df=spark.read.format("json")\
.load("./Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")

- DataFrame은 컬럼을 가지며 스키마로 컬럼을 정의함

In [7]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



- 스키마는 관련된 모든 것을 하나로 묶는 역할을 함

## 5.1. 스키마
- 스키마
  - DataFrame의 컬럼명과 데이터 타입을 정의함
  - 데이터소스에서 스키마를 얻거나 직접 정의할 수 있음

In [9]:
#DataFrame을 생성함
spark.read.format("json")\
    .load("./Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")\
    .schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

- 스키마
  - 여러 개의 StructField 타입 필드로 구성된 StructType 객체
- StructField
  - 이름, 데이터 타입, 컬럼이 값이 없거나 null일 수 있는지 지정하는 불리언값을 가짐
  - 필요한 경우, 컬럼과 관련된 메타데이터를 지정할 수도 있음
- 메타데이터
  - 해당 컬럼과 관련된 정보이며 스파크의 머신러닝 라이브러리에서 사용함
- 스키마는 복합 데이터 타입인 StructType을 가질 수 있음
  - 스파크는 런타임에 데이터 타입이 스키마의 데이터 타입과 일치하지 않으면 오류를 발생시킴

In [11]:
#DataFrame에 스키마를 만들고 적용하는 예제
from pyspark.sql.types import StructField,StructType,StringType,LongType
myManualSchema=StructType([
    StructField("DEST_COUNTRY_NAME",StringType(),True),
    StructField("ORIGIN_COUNTRY_NAME",StringType(),True),
    StructField("count",LongType(),False,metadata={"hello":"world"})
])
df=spark.read.format("json").schema(myManualSchema)\
    .load("./Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")

- 스파크는 자체 데이터 타입 정보를 사용함
  - 프로그래밍 언어의 데이터 타입을 스파크의 데이터 타입으로 설정할 수 없음

## 5.2. 컬럼과 표현식
- 스파크의 컬럼 
  - 스프레드시트, R의 dataframe, Pandas의 DataFrame 컬럼과 유사함
  - 사용자는 표현식으로 DataFrame의 컬럼을 선택, 조작, 제거할 수 있음
  - 표현식을 사용해 레코드 단위로 계산한 값을 단순하게 나타내는 논리적 구조
  - 컬럼의 실젯값을 얻으려면 로우가 필요, 로우를 얻으러면 DataFrame이 필요함
    - DataFrame을 통하지 않으면 외부에서 컬럼에 접근할 수 없음
    - 컬럼 내용을 수정하려면 반드시 DataFrame의 스파크 트랜스포메이션을 사용해야 함


### 5.2.1. 컬럼
- col함수나 column 함수 : 컬럼을 생성하고 참조할 수 있는 간단한 방법
  - 이들 함수는 컬럼명을 인수로 받음

In [12]:
from pyspark.sql.functions import col,column
col("someColumnName")
column("someColumnName")

TypeError: 'str' object is not callable

- 컬럼이 DataFrame에 있을지 없을지는 알 수 없음
  - 컬럼은 컬럼명을 카탈로그에 저장된 정보와 비교하기 전까지 미확인 상태로 남음
  - 분석기가 동작하는 단계에서 컬럼과 테이블을 분석함
  


#### 명시적 컬럼 참조
- DataFrame의 컬럼 : col 메서드로 참조함
  - col 메서드 : 조인 시 유용함
    - ex) DataFrame의 컬럼을 다른 DataFrame의 조인 대상 컬럼에서 참조하기 위해 col 메서드를 사용함
    - 이 메서드를 사용해 명시적으로 컬럼을 정의하면 스파크는 분석기 실행 단계에서 컬럼 확인 절차를 생략함

In [15]:
#df.col("count") -> 이건 scala나 java에서 사용
df["count"] # -> 파이썬에서는 이렇게 써야 한다고 함

Column<'count'>

### 5.2.2. 표현식
- 컬럼은 표현식
- 표현식
  - DataFrame 레코드의 여러 값에 대한 트랜스포메이션 집합을 의미
  - 여러 컬럼명을 입력으로 받아 식별하고 '단일 값'을 만들기 위해 다양한 표현식을 각 레코드에 적용하는 함수라고 생각할 수 있음
    - '단일 값' : Map이나 Array같은 복합 데이터 타입일 수 있음